# Fitting IM Lup

This code runs the MCMC simulation to calculate the best fit parameters for the disk. It uses the logprob function from logprob_parallel.py.

In [ ]:
import shutil
import getpass
import tempfile
from pathlib import Path
from multiprocessing import Pool

import numpy as np
import matplotlib.pyplot as plt

import astropy.units as u
import astropy.constants as c
from astropy.io import fits
import emcee

import dsharp_helper as dh
import dsharp_opac as do
import disklab
from gofish import imagecube

In [ ]:
from helper_functions import get_profile_from_fits
from helper_functions import make_opacs
from helper_functions import chop_forward_scattering
from helper_functions import make_disklab2d_model
from helper_functions import write_radmc3d

In [ ]:
from radmc3dPy import image

In [ ]:
au = c.au.cgs.value
M_sun = c.M_sun.cgs.value
L_sun = c.L_sun.cgs.value
R_sun = c.R_sun.cgs.value

In [ ]:
if getpass.getuser() == 'birnstiel':
    radmc3d_exec = Path('~/.bin/radmc3d').expanduser()
else:
    radmc3d_exec = Path('~/bin/radmc3d').expanduser()

## ALMA data 

set the disk name and get some disk properties from DSHARP

In [ ]:
disk = 'IMLup'
fname_mm_obs = dh.get_datafile(disk)

PA = dh.sources.loc[disk]['PA']
inc = dh.sources.loc[disk]['inc']
distance = dh.sources.loc[disk]['distance [pc]']

clip = 5

lam_mm = 0.125
RMS_jyb = 14e-6

Get the radial profile from the image

In [ ]:
x_mm_obs, y_mm_obs, dy_mm_obs = get_profile_from_fits(
    fname_mm_obs,
    clip=clip,
    inc=inc, PA=PA,
    z0=0.0,
    psi=0.0)

Compare against the DSHARP profile

In [ ]:
ds_prof = dh.get_profile(disk)

f, ax = plt.subplots()
ax.semilogy(x_mm_obs, y_mm_obs)
ax.fill_between(x_mm_obs, y_mm_obs - dy_mm_obs, y_mm_obs + dy_mm_obs, alpha=0.5)

ax.semilogy(ds_prof['r_as'], ds_prof['I_nu'])
ax.fill_between(ds_prof['r_as'], ds_prof['I_nu_l'], ds_prof['I_nu_u'], alpha=0.5)

ax.set_ylim(2e-17, 1e-13);

### SPHERE data

To deproject the scattered light image, we will need to know where the scattering surface is. This is based on the Avenhaus et al. 2018 paper. In `imagecube` this surface can be defined with `z0` and `psi` such that its height $z$ is at

$\mathsf{z = z0 \, \left(\frac{r}{arcsec}\right)^{psi}}\, arcsec$

In [ ]:
z0 = 0.2
psi = 1.27
lam_sca = 1.65e-4

The image does not contain all the required info, so we make a copy of the fits file and modify that one

In [ ]:
fname_sca_obs_orig = 'Qphi_IMLup.fits'
fname_sca_obs = fname_sca_obs_orig.replace('.fits', '_mod.fits')
shutil.copy(fname_sca_obs_orig, fname_sca_obs)

fits.setval(fname_sca_obs, 'cdelt1', value=-3.405e-06)
fits.setval(fname_sca_obs, 'cdelt2', value=3.405e-06)
fits.setval(fname_sca_obs, 'crpix1', value=350.5)
fits.setval(fname_sca_obs, 'crpix2', value=350.5)
fits.setval(fname_sca_obs, 'crval1', value=0.0)
fits.setval(fname_sca_obs, 'crval2', value=0.0)
fits.setval(fname_sca_obs, 'crval3', value=1.65e-4)
fits.setval(fname_sca_obs, 'BUNIT', value='JY/PIXEL')

In [ ]:
# read it with imagecube and derive a radial profile

x_sca_obs, y_sca_obs, dy_sca_obs = get_profile_from_fits(
    fname_sca_obs,
    clip=clip,
    inc=inc, PA=PA,
    z0=z0,
    psi=psi,
    show_plots=True)

## Opacities

define the wavelength, size, and angle grids then calculate opacities and store them in a local file, if it doesn't exist yet.  
**Careful, that takes of the order of >2h**

In [ ]:
%%time
n_lam = 200 # number of wavelength points
n_a = 15 # number of particle sizes
n_theta = 181 # number of angles in the scattering phase function
porosity = 0.3

# wavelength and particle sizes grids

lam_opac = np.logspace(-5, 1, n_lam)
a_opac = np.logspace(-5, 1, n_a)

# make opacities if necessary

opac_dict = make_opacs(a_opac, lam_opac, porosity=porosity, n_theta=n_theta)
fname_opac = opac_dict['filename']

This part chops the very-forward scattering part of the phase function. This part is basically the same as no scattering, but are treated by the code as a scattering event. By cutting this part out of the phase function, we avoid those non-scattering scattering events. This needs to recalculate $\kappa_{sca}$ and $g$.

In [ ]:
fname_opac_chopped = fname_opac.replace('.', '_chopped.')

k_sca_nochop = opac_dict['k_sca']
g_nochop = opac_dict['g']

zscat, zscat_nochop, k_sca, g = chop_forward_scattering(opac_dict)

opac_dict['k_sca'] = k_sca
opac_dict['zscat'] = zscat
opac_dict['g'] = g

rho_s = opac_dict['rho_s']
m = 4 * np.pi / 3 * rho_s * a_opac**3

do.write_disklab_opacity(fname_opac_chopped, opac_dict)

## Emcee part

here we define some inputs and initial parameter sets for the optimization

In [ ]:
# defining number of walkers
nwalkers = 25
ndim     = 7

# setting the priors for some parameters instead of letting them be uniform randoms between (0.1)

sigma_coeff_0   = 10**((np.random.rand(nwalkers)-0.5)*4)
others_0        = np.random.rand(ndim-3,nwalkers)
d2g_coeff_0     = (np.random.rand(nwalkers)+0.5) / 100
d2g_exp_0       = (np.random.rand(nwalkers)-0.5) 

# the input matrix of priors
p0 = np.vstack((sigma_coeff_0,others_0, d2g_coeff_0, d2g_exp_0)).T

# logprob testing

here we test the different steps that need to be taken in the logprob function

In [ ]:
parameters = p0[0, :]
# The different indices in the parameters list correspond to different physical paramters
sigma_coeff = parameters[0]
sigma_exp = parameters[1]
size_exp = parameters[2]
amax_coeff = parameters[3]
amax_exp = parameters[4]
d2g_coeff = parameters[5]
d2g_exp = parameters[6]

In [ ]:
testparameters =[
    7.0,
    0.730,
    0.558,
    0.017,
    0.625,
    0.008,
    0.050,
    ]

create a temporary folder in the current folder

In [ ]:
temp_directory = tempfile.TemporaryDirectory(dir='.')
temp_path = temp_directory.name

set some disk specific parameters (the commented-out values are the ones that were used before)

In [ ]:
# mstar = 0.7 * MS
# lstar = 1.56 * LS
# tstar = 4266.00

mstar = 10.**dh.sources.loc[disk]['log M_star/M_sun'] * M_sun
lstar = 10.**dh.sources.loc[disk]['log L_star/L_sun'] * L_sun
tstar = 10.**dh.sources.loc[disk]['log T_eff/ K']
rstar = np.sqrt(lstar / (4 * np.pi * c.sigma_sb.cgs.value * tstar**4))
PA = dh.sources.loc[disk]['PA']
inc = dh.sources.loc[disk]['inc']
dpc = dh.sources.loc[disk]['distance [pc]']

nr = 100
rin = 0.1 * au
r_c = 300 * au  # ??
rout = 400 * au  # 400au from avenhaus paper  #DSHARP Huang 2018 says 290 au
alpha = 1e-3

### make the disklab 2D model

In [ ]:
disk2d  = make_disklab2d_model(
    testparameters,
    mstar,
    lstar,
    tstar,
    nr,
    alpha,
    rin,
    rout,
    r_c,
    fname_opac_chopped,
    show_plots=True
)

In [ ]:
print(f'disk to star mass ratio = {disk2d.disk.mass / disk2d.disk.mstar:.2g}')

In [ ]:
write_radmc3d(disk2d, lam_opac, temp_path, show_plots=True)

## Calculate the mm continuum image

In [ ]:
fname_mm_sim = Path(temp_path) / 'image_mm.fits'
disklab.radmc3d.radmc3d(
    f'image incl {inc} posang {PA-90} npix 500 lambda {lam_mm * 1e4} sizeau {2 * rout / au} secondorder  setthreads 1',
    path=temp_path,
    executable=str(radmc3d_exec)
    )

In [ ]:
radmc_image = Path(temp_path) / 'image.out'
if radmc_image.is_file():
    im_mm_sim = image.readImage(radmc_image)
    radmc_image.replace(Path(temp_path) / 'image_mm.out')
    im_mm_sim.writeFits(str(fname_mm_sim), dpc=dpc, coord='15h56m09.17658s -37d56m06.1193s')

Read in the fits files into imagecubes, and copy the beam information from the observation to the simulation.

In [ ]:
iq_obs = imagecube(str(fname_mm_obs))

iq_sim = imagecube(str(fname_mm_sim))
iq_sim.bmaj, iq_sim.bmin, iq_sim.bpa = iq_obs.beam
iq_sim.beamarea_arcsec = iq_sim._calculate_beam_area_arcsec()
iq_sim.beamarea_str = iq_sim._calculate_beam_area_str()

In [ ]:
im_cgs_sim = iq_sim.data * iq_sim.pix_per_beam / iq_sim.beamarea_str * 1e-23
im_cgs_obs = iq_obs.data / iq_sim.beamarea_str * 1e-23

f, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
vmin = RMS_jyb * 1e-23 / iq_obs.beamarea_str # the RMS from dsharp (Jy/beam) to CGS conversion
vmax = 20 * vmin
ax[0].imshow(im_cgs_sim, extent=iq_sim.extent, vmin=vmin, vmax=vmax, origin='lower')
ax[1].imshow(im_cgs_obs, extent=iq_obs.extent, vmin=vmin, vmax=vmax, origin='lower')
ax[0].set_xlim([2, -2])
ax[0].set_ylim([-2, 2]);

ax[0].axis('off')
ax[1].axis('off')
f.subplots_adjust(wspace=0)
f.savefig('mm.pdf', transparent=True, bbox_inches='tight')

In [ ]:
x_mm_sim, y_mm_sim, dy_mm_sim = get_profile_from_fits(
    str(fname_mm_sim),
    clip=clip,
    inc=inc, PA=PA,
    z0=0.0,
    psi=0.0,
    beam=iq_obs.beam)

here we estimate the noise of the azimuthally averaged profile by dividing the RMS noise of the image by the approximate number of beams along the annulus

In [ ]:
vmin_avg = vmax / (2 * np.pi * x_mm_obs * np.sqrt(iq_obs.beam[0] * iq_obs.beam[1]) / iq_obs.beamarea_arcsec)

Plot the azimuthal profile and error estimate

In [ ]:
f, ax = plt.subplots(dpi=150)
ax.semilogy(x_mm_obs, y_mm_obs, label='ALMA data')
ax.fill_between(x_mm_obs, y_mm_obs - dy_mm_obs, y_mm_obs + dy_mm_obs, alpha=0.5)

ax.semilogy(x_mm_sim, y_mm_sim, label='model')
ax.fill_between(x_mm_sim, y_mm_sim - dy_mm_sim, y_mm_sim + dy_mm_sim, alpha=0.5)

#ax.fill_between(x_mm_obs, y_mm_obs - (vmax * err_est), y_mm_obs + (vmax * err_est), alpha=0.5)

ax.axhline(vmin, c='0.5', ls='--', label='image RMS noise')
ax.semilogy(x_mm_obs, vmin_avg, c='k', ls='--', label='expected RMS noise of profile')

ax.semilogy(x_mm_obs, np.maximum(y_mm_obs, vmin_avg), c='k', ls='-')

ax.set_xlim(1.5, 2.5);
ax.set_ylim(5e-17, 1e-13)
ax.set_xlabel('r [arcsec]')
ax.set_ylabel('Intensity [erg/(s cm$^2$ Hz sr)]')
ax.legend(fontsize='small')
f.savefig('profile_mm.pdf', transparent=True, bbox_inches='tight')

In [ ]:
i_max = max(len(x_mm_obs), len(x_mm_sim))

x_mm_sim = x_mm_sim[:i_max]
y_mm_sim = y_mm_sim[:i_max]
x_mm_obs = x_mm_obs[:i_max]
y_mm_obs = y_mm_obs[:i_max]

if not np.allclose(x_mm_sim, x_mm_obs):
    raise AssertionError('observed and simulated radial profile grids are not equal')

Calculate the log probability for the mm here

In [ ]:
log_prob_mm = -0.5 * np.sum((np.interp(observed_radius, radial / 158,
                                       radial_profile) - observed_intensity)**2 / (observed_intensity_error**2)) / len(observed_radius)

# Scattered light

In [ ]:
# CUT OUT OPACITIES PART 2
from disklab.radmc3d import write
import dsharp_opac as opacity

#for p in Path(temp_path).glob('dustkappa_*.inp'):
#    p.unlink()

for i_grain in range(n_a):
    opacity.write_radmc3d_scatmat_file(i_grain, opac_dict, f'{i_grain}', path=temp_path)

In [ ]:
with open(Path(temp_path) / 'dustopac.inp', 'w') as f:
    write(f, '2               Format number of this file')
    write(f, '{}              Nr of dust species'.format(n_a))

    for i_grain in range(n_a):
        write(f, '============================================================================')
        write(f, '10               Way in which this dust species is read')
        write(f, '0               0=Thermal grain')
        write(f, '{}              Extension of name of dustscatmat_***.inp file'.format(i_grain))

    write(f, '----------------------------------------------------------------------------')

# image calculation
disklab.radmc3d.radmc3d(f'image incl {inc} posang {PA-90} npix 500 lambda {lam_sca / 1e-4} sizeau {2 * rout / au} setthreads 4', path=temp_path)


In [ ]:
fname_sca_sim = Path(temp_path) / 'image_sca.fits'
if (Path(temp_path) / 'image.out').is_file():
    (Path(temp_path) / 'image.out').replace(fname_sca_sim.with_suffix('.out'))

In [ ]:
im = image.readImage(fname_sca_sim.with_suffix('.out'))
im.writeFits(str(fname_sca_sim), dpc=dpc, coord='15h56m09.17658s -37d56m06.1193s')

In [ ]:
iq_obs = imagecube(str(fname_sca_obs))
iq_sim = imagecube(str(fname_sca_sim))

In [ ]:
x_sca_sim, y_sca_sim, dy_sca_sim = get_profile_from_fits(
    str(fname_sca_sim),
    clip=clip,
    inc=inc, PA=PA,
    z0=z0,
    psi=psi)

In [ ]:
f, ax = plt.subplots()
ax.semilogy(x_sca_obs, y_sca_obs)
ax.fill_between(x_sca_obs, y_sca_obs - dy_sca_obs, y_sca_obs + dy_sca_obs, alpha=0.5)

ax.semilogy(x_sca_sim, y_sca_sim)
ax.fill_between(x_sca_sim, y_sca_sim - dy_sca_sim, y_sca_sim + dy_sca_sim, alpha=0.5)

ax.set_ylim(1e-17, 1e-13);

In [ ]:
im_cgs_sim = iq_sim.data
im_cgs_obs = iq_obs.data

f, ax = plt.subplots(1, 2, figsize=(10, 5), sharex=True, sharey=True)
vmin = 5e-8
vmax = 20 * vmin
ax[0].imshow(im_cgs_sim, extent=iq_sim.extent, vmin=vmin, vmax=vmax, origin='lower')
ax[1].imshow(im_cgs_obs, extent=iq_obs.extent, vmin=vmin, vmax=vmax, origin='lower')
ax[0].set_xlim([2, -2])
ax[0].set_ylim([-2, 2])

ax[0].axis('off')
ax[1].axis('off')
f.subplots_adjust(wspace=0)
f.savefig('sca.pdf', transparent=True, bbox_inches='tight')

**TODO**

- [x] need to bring the mm profiles on the same scale to compare 
- [ ] figure out units of scattered light image (flux-calibrated?)
- [ ] implement "40 mas beam" for the weighting of the SPHERE profiles
- [ ] figure out how to handle the noisy parts
- [ ] calculate the logP from the profiles
- [ ] figure out how to treat the SPHERE image asymmetry (forward scattering)
- [ ] test effect of porosity on asymmetry
- [ ] whatever is below

In [ ]:
i_max = min(len(x_sca_obs), len(x_sca_sim))

x_sca_sim = x_sca_sim[:i_max]
y_sca_sim = y_sca_sim[:i_max]
x_sca_obs = x_sca_obs[:i_max]
y_sca_obs = y_sca_obs[:i_max]

if not np.allclose(x_sca_sim, x_sca_obs):
    raise AssertionError('observed and simulated radial profile grids are not equal')

### calculate $\log P$

In [ ]:
log_prob_scat = -0.5 * np.nansum((np.interp(x, sim_x, sim_profile) - profile)**2 / (profile_err**2)) / len(x)

# adding the two log probs and then multiplying by a large factor in order to make the MCMC more sensitive to changes
log_prob = (log_prob_mm + log_prob_scat)

<hr>

**Here comes the rest of `MCMC_parallelized.py`, not cleaned up yet**

In [ ]:
print('step1')

# Parallelizing the simluation and running it for 250 iterations
with Pool(processes=100) as pool:
    sampler1 = emcee.EnsembleSampler(nwalkers, ndim, logprob, args=[profile, profile_err, x_arcsec], pool=pool)
    sampler1.run_mcmc(p0, 250)

print(sampler1.iteration)    

print('step2')
sampler2 = deepcopy(sampler1)
sampler2.log_prob_fn = None
with open('sampler.pickle', 'wb') as fid:
    pickle.dump(sampler2, fid)